In [ ]:
import os
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
from google.colab import drive
import crapaudset as cs


drive.mount("/content/drive",force_remount=True)


Mounted at /content/drive


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 249060, done.
remote: Counting objects: 100% (1750/1750), done.
remote: Compressing objects: 100% (629/629), done.
remote: Total 249060 (delta 1597), reused 1206 (delta 1118), pack-reused 247310
Receiving objects: 100% (249060/249060), 502.54 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (226510/226510), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [ ]:
def get_instance_model(num_classes = 5):
    # num_classes which is user-defined

    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="DEFAULT")
    for param in model.parameters():
      param.requires_grad = False
    # replace the classifier with a new one, that has
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


model = get_instance_model()

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    # transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)
def collate_fn(batch):
    return tuple(zip(*batch))


In [ ]:
from engine import train_one_epoch, evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # use our dataset and defined transformations
    dataset = cs.CrapaudSet("/content/drive/MyDrive/Colab Notebooks/crapaud/path_and_bounding_box.csv","/content/drive/MyDrive/Colab Notebooks/crapaud/subset", get_transform(train=True))
    dataset_test = cs.CrapaudSet("/content/drive/MyDrive/Colab Notebooks/crapaud/path_and_bounding_box.csv","/content/drive/MyDrive/Colab Notebooks/crapaud/subset", get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_instance_model()

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [ ]:
main()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [  0/286]  eta: 0:19:47  lr: 0.000023  loss: 1.6817 (1.6817)  loss_classifier: 1.5524 (1.5524)  loss_box_reg: 0.1011 (0.1011)  loss_objectness: 0.0264 (0.0264)  loss_rpn_box_reg: 0.0017 (0.0017)  time: 4.1515  data: 1.6911  max mem: 1149
Epoch: [0]  [ 10/286]  eta: 0:03:03  lr: 0.000198  loss: 1.6342 (1.5799)  loss_classifier: 1.4689 (1.4392)  loss_box_reg: 0.0305 (0.0443)  loss_objectness: 0.0829 (0.0909)  loss_rpn_box_reg: 0.0049 (0.0054)  time: 0.6640  data: 0.1688  max mem: 1149
Epoch: [0]  [ 20/286]  eta: 0:02:13  lr: 0.000373  loss: 1.2668 (1.2390)  loss_classifier: 1.1320 (1.1015)  loss_box_reg: 0.0300 (0.0332)  loss_objectness: 0.0829 (0.0981)  loss_rpn_box_reg: 0.0055 (0.0061)  time: 0.3192  data: 0.0182  max mem: 1149
Epoch: [0]  [ 30/286]  eta: 0:01:53  lr: 0.000548  loss: 0.4733 (0.9386)  loss_classifier: 0.3032 (0.8044)  loss_box_reg: 0.0092 (0.0293)  loss_objectness: 0.0733 (0.0990)  loss_rpn_box_reg: 0.0043 (0.0060)  time: 0.3207  data: 0.0195  max mem: 1149


In [ ]:
torch.save(model, "/content/drive/MyDrive/Colab Notebooks/crapaud/model.pt")

#model_scripted = torch.jit.script(model) # Export to TorchScript
#model_scripted.save('/content/drive/MyDrive/Colab Notebooks/crapaud/model_torchscript.pt') # Save

In [ ]:
model = torch.load("/content/drive/MyDrive/Colab Notebooks/crapaud/model.pt")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

img_pil = Image.open("/content/triton.jpeg").convert("RGB")
img_tensor = T.ToTensor()(img_pil)
model.eval()

res = model([img_tensor])
res

[{'boxes': tensor([[1.0262e+02, 0.0000e+00, 5.9479e+02, 2.9044e+02],
          [4.5784e+01, 0.0000e+00, 3.1720e+02, 2.8391e+02],
          [3.7559e+00, 0.0000e+00, 7.3738e+02, 4.5509e+02],
          [8.5683e+02, 0.0000e+00, 1.0913e+03, 2.3610e+02],
          [6.6120e+02, 2.5051e+02, 6.7100e+02, 2.6584e+02],
          [4.1193e+02, 2.3009e+02, 1.2800e+03, 5.1707e+02],
          [7.3851e+01, 0.0000e+00, 3.6983e+02, 4.5799e+02],
          [9.1804e+00, 2.4951e+02, 1.0285e+03, 4.9169e+02],
          [9.1500e+02, 1.3925e+02, 9.7859e+02, 1.9797e+02],
          [7.2831e+02, 0.0000e+00, 1.1736e+03, 2.1361e+02],
          [1.1634e+03, 2.0046e+02, 1.1996e+03, 2.4046e+02],
          [3.2051e+02, 2.5525e+02, 1.1227e+03, 6.6341e+02],
          [6.2471e+02, 2.8552e+02, 1.2800e+03, 6.6396e+02],
          [1.1667e+03, 5.1677e+02, 1.2088e+03, 5.6953e+02],
          [1.1556e+03, 2.2725e+02, 1.2023e+03, 2.4907e+02],
          [6.5701e+02, 3.1306e+02, 1.2800e+03, 4.6360e+02],
          [5.8646e+02, 6.8180e+

In [ ]:
bbox_t = res[0]['boxes'][0]

bbox_t = bbox_t.int().to(torch.torch.uint8)
bbox_t

tensor([102,   0,  82,  34], dtype=torch.uint8)

In [ ]:
img_tensor.mul(255).int().to(torch.torch.uint8)

tensor([[[36, 36, 35,  ..., 38, 39, 40],
         [36, 36, 35,  ..., 36, 37, 38],
         [36, 36, 35,  ..., 37, 37, 37],
         ...,
         [36, 35, 35,  ..., 20, 20, 19],
         [36, 35, 35,  ..., 20, 19, 19],
         [36, 35, 35,  ..., 19, 19, 19]],

        [[36, 36, 35,  ..., 38, 39, 40],
         [36, 36, 35,  ..., 36, 37, 38],
         [36, 36, 35,  ..., 37, 37, 37],
         ...,
         [36, 35, 35,  ..., 20, 20, 19],
         [36, 35, 35,  ..., 20, 19, 19],
         [36, 35, 35,  ..., 19, 19, 19]],

        [[36, 36, 35,  ..., 38, 39, 40],
         [36, 36, 35,  ..., 36, 37, 38],
         [36, 36, 35,  ..., 37, 37, 37],
         ...,
         [36, 35, 35,  ..., 20, 20, 19],
         [36, 35, 35,  ..., 20, 19, 19],
         [36, 35, 35,  ..., 19, 19, 19]]], dtype=torch.uint8)

In [ ]:
img = torchvision.utils.draw_bounding_boxes(img_tensor.mul(255).int().to(torch.torch.uint8), bbox_t, width=2)
img = T.ToPILImage()(img.to('cpu'))
plt.imshow(img)

TypeError: ignored